In [1]:
from socket import *
from time import ctime
import signal, os

In [2]:
CM_SUBSCRIBE = 10
CM_NEW_ROUND = 11
CM_CATEGORY = 12
CM_CATEGORY_RECV = 13
CM_RING = 14
CM_ANSWER = 15
CM_ANSWER_SHOW = 16

# MAJORSPLIT = "::"
# MINORSPLIT = ";"
SPLIT = "::"

SM_WELCOME = 10
SM_NEW_GAME = 11
SM_NEW_ROUND = 12
SM_CATEGORY = 13
SM_QUESTION = 14
SM_RING_CLIENT = 15
SM_ANSWER = 16

JOINING_GAME = 0
GAME_IN_PROGRESS = 1
ROUND_IN_PROGRESS = 2
CATEGORY_SELECTED = 3
DISPLAY_QUESTION = 4
WAIT_FOR_ANSWER = 5
ANSWER_SHOW = 6

PlayerId = -1
PlayerName = ""
timeout = 10
PointsPerQues = -1
CateNames = []
PlayerState = JOINING_GAME


In [3]:
# def writeMsg(RecvMsg, txt1, txt2):
# #     msg = str(RecvMsg) + MAJORSPLIT + txt1 + MAJORSPLIT + txt2
#     msg = str(RecvMsg) + SPLIT + txt1 + SPLIT + txt2


In [4]:
import signal, os
def readMsg(msg):
    global PlayerId
    global PlayerName
    global PlayerState
    global CateNames
    global timeout
    global PointsPerQues
    field = msg.split(SPLIT)
    RecvMsg = int(field[0])    
    if RecvMsg == SM_WELCOME and PlayerState == JOINING_GAME:
        print("Please enter your name:")
        PlayerName = input("Name: ")
        PlayerState = GAME_IN_PROGRESS
        msg = str(CM_SUBSCRIBE) + SPLIT + PlayerName + SPLIT + "#"
    
    if RecvMsg == SM_NEW_GAME and PlayerState == GAME_IN_PROGRESS:
        NumOfPlayers = int(field[1])
        PlayersNames = field[2].split()
        NumOfCategories = int(field[3])
        CateNames = field[4].split()
        PointsPerQues = int(field[5])
        timeout = int(field[6])
        print("Hello, everyone! Are you ready? Game Start!")
        print("These are basic informations for our players:")
        print("**********"*10)
        print("There are %d players in the game:" % NumOfPlayers)
        for i, playername in enumerate(PlayersNames):
            print("Player: %s, Id: %d" % (playername, i+1))
        print("%d categories:" % NumOfCategories)
        for i, categoryname in enumerate(CateNames):
            print("Category number - %d: %s" % (i, categoryname))
        print("Each category has two questions and each question worth %d points" % PointsPerQues)
        print("You have %d seconds to ring the bell if you know the answer" % timeout)
        PlayerId = PlayersNames.index(PlayerName) + 1
        PlayerState = ROUND_IN_PROGRESS
        msg = str(CM_NEW_ROUND) + SPLIT + str(PlayerId) + SPLIT + "#"
    if RecvMsg == SM_NEW_ROUND and PlayerState in (ROUND_IN_PROGRESS, ANSWER_SHOW):
        ClientId = int(field[1])
        print("Let's start a new round!")
        print("**********"*10)
        if ClientId == PlayerId:
            print("Please select a category:")
            CateId = input("Category: ")
            msg = str(CM_CATEGORY) + SPLIT + CateId + SPLIT + str(PlayerId)
        else:
            print("Wait for Player %d select" % PlayerId)
            print("----------"*10)
            msg = str(CM_CATEGORY) + SPLIT + "-1" + SPLIT + str(PlayerId)
        PlayerState = CATEGORY_SELECTED
    if RecvMsg == SM_CATEGORY and PlayerState == CATEGORY_SELECTED:
        CateId = int(field[1])
        CateName = CateNames[CateId]
        print("In this round, the selected category is: %s" % CateName)
        PlayerState = DISPLAY_QUESTION
        msg = str(CM_CATEGORY_RECV) + SPLIT + str(CateId) + SPLIT + "#"
    if RecvMsg == SM_QUESTION and PlayerState == DISPLAY_QUESTION:
        question = field[1]
        print("The question is: %s" % question)
        print("(If you know the answer, press the ring button quickly\n and remember you have 10 miliseconds to ring the bell).")
        print("Timer start!")
        print("**********"*10)
        class TimeoutError(Exception):
            pass
        def interrupted(signum, frame):
            raise TimeoutError
        signal.signal(signal.SIGALRM, interrupted)
        signal.alarm(3)
        try:
            answer = input("Please ring the bell(enter your ID) if you know the answer: ")
        except TimeoutError:
            print("\ntimeout!")
            answer = ""
        signal.alarm(0)  # 读到输入的话重置信号
        if answer != "":
            PlayerState = DISPLAY_QUESTION
            msg = str(CM_RING) + SPLIT + answer + SPLIT + "#"
        else:
            PlayerState = WAIT_FOR_ANSWER
            msg = str(CM_RING) + SPLIT + "#" + SPLIT + "#"
    if RecvMsg == SM_RING_CLIENT and PlayerState == DISPLAY_QUESTION:
        RingPlayerId = int(field[1])
        if PlayerId == RingPlayerId:
            print("**********"*6)
            print("Congratulations! player-%d get the chance to answer this question." % PlayerId)
            print("**********"*6)
            answer = input("Please show your answer: (if you don't know the answer, please enter 'no')")
            PlayerState = ANSWER_SHOW
            msg = str(CM_ANSWER) + SPLIT + answer + SPLIT + "#"
        else:
            print("**********"*6)
            print("Sorry, player %s ring the bell first and you need to wait, you cannot answer the question..." % RingPlayerId)
            PlayerState = ANSWER_SHOW
            msg = str(CM_ANSWER) + SPLIT + "#" + SPLIT + "#"
    if RecvMsg == SM_ANSWER and PlayerState == ANSWER_SHOW:
        PlayerAnswer = field[1]
        CorrAnswer = field[2]
        print("Player's answer is: %s and the correct answer is: %s" % (PlayerAnswer, CorrAnswer))
        print("**********"*6)
        if PlayerAnswer == CorrAnswer:
            print("Good job! Win %s points!" % PointsPerQues)
        else:
            print("Sorry! Wrong answer!")
        PlayerState = ANSWER_SHOW
        msg = str(CM_ANSWER_SHOW) + SPLIT + "#" + SPLIT + "#"
    return msg
       

In [5]:
serverName = '127.0.0.1'
serverPort = 33333
clientSocket = socket(AF_INET, SOCK_STREAM)
clientSocket.connect((serverName, serverPort))
while True:
    msg = clientSocket.recv(1024).decode()
    outmsg = readMsg(msg)
    clientSocket.send(outmsg.encode())
clientSocket.close()

In [6]:
# x = readMsg("10")
# print("*****", x)
# print("_"*70)

Please enter your name:
Name: bbb
***** 10::bbb::#
______________________________________________________________________


In [7]:
# x = readMsg("11::2::aaa bbb::2::math cs::500::1000")
# print("****", x)
# print("*** PlayerId %d" % PlayerId)
# print("_"*70)

Hello, everyone! Are you ready? Game Start!
These are basic informations for our players:
************************************************************
There are 2 players in the game:
Player: aaa, Id: 1
Player: bbb, Id: 2
2 categories:
Category number - 0: math
Category number - 1: cs
Each category has two questions and each question worth 500 points
You have 1000 seconds to ring the bell if you know the answer
**** 11::2::#
*** PlayerId 2
______________________________________________________________________


In [8]:
# x = readMsg("12::1")
# print("***", x)
# print("_"*70)

Let's start a new round!
************************************************************
Wait for Player 2 select
*** 12::#::2
______________________________________________________________________


In [9]:
# x = readMsg("13::1")
# print("***", x)
# print("_"*70)

In this round, the selected category is: cs
*** 13::1::#
______________________________________________________________________


In [10]:
# x = readMsg("14::what is 1011?")
# print("***", x)
# print("_"*70)


The question is: what is 1011?
(If you know the answer, press the ring button quickly
 and remember you have 5 miliseconds to ring the bell).
Timer start!
************************************************************
Please ring the bell(enter your ID) if you know the answer: 2
*** 14::2::#
______________________________________________________________________


In [11]:
# x = readMsg("15::1")
# print("***", x)
# print("_"*70)

Congratulations! player: 1 get the chance to answer this question.
Please show your answer: 9
*** 15::9::#
______________________________________________________________________


In [12]:
# x = readMsg("16::11::9")
# print("***", x)
# print("_"*70)

The player's answer is: 11 and the correct answer is: 9
*** 16::#::#
______________________________________________________________________
